## Create cell nucleus layer for Neuroglancer

This script reads a CSV file of cell nucleus positions and sizes, and outputs JSON data that can be pasted into Neuroglancer as one of the layers in its state.

In [ ]:
csv_file = "/home/joe/zheng-mouse-hippo/nucleus-centroids-filtered.csv"
data_resolution = (384, 384, 45)

In [ ]:
with open(csv_file, 'r') as f:
    lines = f.readlines()
data = []
header = None
for line in lines:
    if header is None:
        header = line.strip().split(',')
    else:
        data.append(list(map(int, line.split(','))))
print("Got such data as:")
print(header)
for line in data[:10]:
    print(line)

In [ ]:
def calc_radius(voxels):
    """Calculate the (x, y, z) radius, in linear voxels, given volume in cubic voxels."""
    # Hard-coded for now:
    r = 5
    return (r, r, r * data_resolution[0] / data_resolution[2])

In [ ]:
header = '''
    {
      "type": "annotation",
      "source": {
        "url": "local://annotations",
        "transform": {
          "outputDimensions": {"x": [12,"nm"], "y": [12,"nm"], "z": [45, "nm"]},
          "inputDimensions": {"x": [#IRX,"nm"], "y": [#IRY,"nm"], "z": [#IRZ, "nm"]}
        }
      },
      "tab": "annotations",
      "annotations": [
'''
entry = '''        {
          "center": [ #CX, #CY, #CZ ],
          "radii": [ #RX, #RY, #RZ ],
          "type": "ellipsoid",
          "id": "#ID",
          "description": "#NOTE"
        }'''
footer = '''
      ],
      "name": "nuclei"
    }
'''

In [ ]:
def subfields(s, field_map):
    for k,v in field_map.items():
        s = s.replace('#' + k, str(v))
    return s

In [ ]:
print(subfields(header, {'IRX':data_resolution[0], 'IRY':data_resolution[1], 'IRZ':data_resolution[2]}))
first = True
for line in data:
    r = calc_radius(line[4])
    if not first:
        print(',')
    print(subfields(entry, { 
        'ID': '39842' + str(line[0]),
        'CX': line[1], 'CY': line[2], 'CZ': line[3], 
        'RX': r[0], 'RY': r[1], 'RZ': r[2],
        'NOTE': str(line[4]) + " voxels"
        }), end='')
    first = False
print(footer)